In [2]:
import prettytable as prettytable
import random as rnd
from random import choice
import numpy as np
import pandas as pd 

eliteSchedules = 1
mutationRate = 0.1
rouletteSelection = 2
population_size = 5

class InputData:
    def __init__(self):    
        self._rooms = ['C301', 'C302', 'C303' , 'C304' , 'C305' , 'C306' , 'C307' , 'C308' , 'C309' , 'C310']
        self._examTime = ['9:00-12:00','2:00-5:00']
        self._days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        
        self._numberOfClasses = 10
        self._numberOfStudentsPerClass = 28
        self._examDuration = 3
        
        #....................function that reads data from given files....................
        
        def readcsv(filename):
            data = pd.read_csv(filename)
            return(np.array(data))

        #.....................reading input from given files.............................
        
        self._stud = readcsv("studentNames.csv")
        self._teach = readcsv("teachers.csv")
        self._studcourses = readcsv("studentCourse.csv")
        self._cour = readcsv("courses.csv")
        
        #......................arrays to store data later...............................
        
        self._student = [None]*len(self._studcourses)
        self._studentCourse = [None]*len(self._studcourses)
        self._teachers = [None]*len(self._teach)
        self._course = [None]*len(self._cour)
        self._courseCode = [None]*len(self._cour)

        #......................store Teachers into an array.............................
        
        for i in range (0,len (self._teach)):
            self._teachers[i]  = self._teach[i][0]

        #...............store Students and their respective Courses in Student Class Object.................
        
        self._Student = []
        for i in range (0,len (self._studcourses)):
            self._student[i]=self._studcourses[i][0]
            self._studentCourse[i]=self._studcourses[i][1]
            self._Student.append(Student(self._student[i],self._studentCourse[i]))
   
        #................store Course, its respective Code and room in Course Class Object.................
        
        self._Course = []
        for i in range (0,len (self._cour)):
            self._courseCode[i]=self._cour[i][0]
            self._course[i] =self._cour[i][1]
            room = self._rooms[rnd.randrange(0, len(self._rooms))]
            self._Course.append(Course(self._course[i],self._courseCode[i],room))
                   
    def getDay(self):
        return self._days
    def getRoom(self):
        room = self._rooms[rnd.randrange(0, len(self._rooms))]
        return room
    def getTeacher(self):
        teach = self._teachers[rnd.randrange(0, len(self._teachers))]
        return teach
    def getCourse(self):
        return self._Course
    def getExamTime(self):
        time = self._examTime[rnd.randrange(0, len(self._examTime))]
        return time
    def getNumberOfClasses(self):
        return self._numberOfClasses
    def getStudent(self):
        return self._Student
    def getstudentCourse(self):
        return self._studentCourse


class Schedule:
    def __init__(self):
        self._data = data
        self._exams = []
        self._conflicts = 0
        self._fitness = -1
        self._examNum = 0
        self._isFitnessChanged = True
        self._sorted = []
    def getExam(self):
        self._isFitnessChanged = True
        return self._exams
    def getConflict(self):
        return self._conflicts
    
    #.................if fitness changes , calculate and return new fitness...................
    def getFitness(self):
        if (self._isFitnessChanged == True):
            self._fitness = self.calculateFitness()
            self._isFitnessChanged = False
        return self._fitness
    
    #.................initializing exams......................
    def initialize(self):
        days = self._data.getDay()
        for i in range(0, len(days)):
            timeSlots = self._data.getExamTime()
            courses = self._data.getCourse()
            for j in range(0, len(timeSlots)):
                temp = rnd.randrange(0, len(courses))
                newExam = Exam(self._examNum, courses[temp] , courses[temp]._code)
                self._examNum += 1
                newExam.setDay(days[rnd.randrange(0,len(days))])
                newExam.setExamTime(data.getExamTime())
                newExam.setRoom(courses[j].getAllowedRooms()[rnd.randrange(0, len(courses[j].getAllowedRooms()))]) 
                self._exams.append(newExam)
        return self
    
    #.............sorting exams according to time and day................
    def sortDays(self):
        exams = self.getExam()
        
        firstHour = []
        secondHour = []
 
        monTemp =[]
        tuesTemp = []
        wedTemp = []
        thursTemp = []
        friTemp = []
        
        #...............seperating shifts of exams ................
        
        for i in range(0, len(exams)):
            if exams[i].getExamTime() == '9:00-12:00':
                firstHour.append(exams[i])
            elif exams[i].getExamTime() == '2:00-5:00':
                secondHour.append(exams[i])
        
        #..............seperating morning shift exams on basis of exam day...............
        
        for i in range(0, len(firstHour)):
            if firstHour[i].getDay() == 'Monday':
                monTemp.append(firstHour[i])
            elif firstHour[i].getDay() == 'Tuesday':
                tuesTemp.append(firstHour[i])
            elif firstHour[i].getDay() == 'Wednesday':
                wedTemp.append(firstHour[i])
            elif firstHour[i].getDay() == 'Thursday':
                thursTemp.append(firstHour[i])
            elif firstHour[i].getDay() == 'Friday':
                friTemp.append(firstHour[i])
          
        #..............seperating evening shift exams on basis of exam day...............
        
        for i in range(0, len(secondHour)):
            if secondHour[i].getDay() == 'Monday':
                monTemp.append(secondHour[i])
            elif secondHour[i].getDay() == 'Tuesday':
                tuesTemp.append(secondHour[i])
            elif secondHour[i].getDay() == 'Wednesday':
                wedTemp.append(secondHour[i])
            elif secondHour[i].getDay() == 'Thursday':
                thursTemp.append(secondHour[i])
            elif secondHour[i].getDay() == 'Friday':
                friTemp.append(secondHour[i])
 
        self._sorted = [monTemp, tuesTemp, wedTemp, thursTemp, friTemp]
        return self   
    
    def getSorted(self):
        return self._sorted
    
    #...................checking if a student has 2 exams at the same time and day......................
    def examConflict(self):
        stud= data.getStudent()
        exam = self.getExam()
        brk1=0
        brk2=0
        for i in range(0, len(exam)):
            for j in range(i+1, len(exam)):
                if exam[i].getDay()==exam[j].getDay() and exam[i].getExamTime() == exam[j].getExamTime():
                    if(brk1==0):
                        for k in range (0 , len(stud)):
                            if stud[k].getCourse()== exam[i].getcourseCode():
                                for l in range (k+1 , len(stud)):
                                    if stud[l].getCourse()== exam[j].getcourseCode():
                                        if stud[k].getName()==stud[l].getName() and stud[k].getCourse()!=stud[l].getCourse():
                                            if(brk2==0):
                                                brk1+=1
                                                brk2+=1
                                                return True
        return False
    
    #....................calculating fitness...............................
    def calculateFitness(self):
        self.sortDays()
        self._conflicts = 0
        fl = 0
        exams = self.getExam()
        for i in range(0, len(exams)):
            if self.examConflict() == True:
                if fl==0:
                    self._conflicts += 0.5
                    fl+=1
            for j in range(0, len(exams)):
                if (j >= i):
                    
                    #.........................teacher can not invigilate 2 exams at the same time...............
                    
                    if ((exams[i].getExamTime() == exams[j].getExamTime() and exams[i].getDay() == exams[j].getDay()) and exams[i].getID() != exams[j].getID()):
                        if exams[i].getTeachers() == exams[j].getTeachers():
                            self._conflicts += 1
                            
                    #..........................teacher can not invigilate 2 exams in a row.....................
                    
                    if ((exams[i].getExamTime() != exams[j].getExamTime() and exams[i].getDay() == exams[j].getDay()) and exams[i].getID() != exams[j].getID()):
                        if exams[i].getTeachers() == exams[j].getTeachers():
                            self._conflicts += 1
        if self._conflicts==0.5:
            self._conflicts=0
        return 1 / (1.0*self._conflicts + 1)
    
    def __str__(self):
        returnValue = ""
        for i in range(0, len(self._exams)-1):
            returnValue += str(self._exams[i]) + ", "
        returnValue += str(self._exams[len(self._exams) - 1])
        return returnValue
    
class Population:
    def __init__(self, size):
        self._size = size
        self._data = data
        self._schedules = []
        for i in range(0, size):
            self._schedules.append(Schedule().initialize())
    def getSchedules(self):
        return self._schedules
       
class GeneticAlgorithm:
    def evolve(self, population):
        return self.mutatePopulation(self.crossoverPopulation(population))

    #..............performing crossover on population.....................
    def crossoverPopulation(self, pop):
        crossoverPopul = Population(0)
        for i in range(eliteSchedules):
            crossoverPopul.getSchedules().append(pop.getSchedules()[i])
        i = eliteSchedules
        while i < population_size:
            schedule1 = self.roulettePopulation(pop).getSchedules()[0]
            schedule2 = self.roulettePopulation(pop).getSchedules()[0]
            crossoverPopul.getSchedules().append(self.crossoverSchedule(schedule1, schedule2))
            i += 1
        return crossoverPopul

    #..............performing mutation on population.....................
    def mutatePopulation(self, population):
        for i in range(eliteSchedules, population_size):
            self.mutateSchedule(population.getSchedules()[i])
        return population

    #..............performing crossover on population.....................
    def crossoverSchedule(self, schedule1, schedule2):
        crossoverSch = Schedule().initialize()
        for i in range(0, min(len(crossoverSch.getExam()) , len(schedule1.getExam()) , len(schedule2.getExam()))):
            if (rnd.random() > 0.5):
                crossoverSch.getExam()[i] = schedule1.getExam()[i]
            else:
                crossoverSch.getExam()[i] = schedule2.getExam()[i]
        return crossoverSch

    #..............performing mutation on population.....................
    def mutateSchedule(self, mSchedule):
        schedule = Schedule().initialize()
        for i in range(0, min(len(mSchedule.getExam()),len(schedule.getExam()) )):
            if (rnd.random() < mutationRate):
                mSchedule.getExam()[i] = schedule.getExam()[i]
        return mSchedule

    #..............roulette population.....................
    def roulettePopulation(self, pop):
        roulettePop = Population(0)
        sch = pop.getSchedules()
        sum = 0
        for i in range(0, len(sch)):
            sum += sch[i].getFitness()
        pick    = rnd.uniform(0, sum)
        choice = 0
        j = 0
        for i in range(0, len(sch)):
            choice += sch[i].getFitness()
            if choice > pick and j < rouletteSelection:
                roulettePop.getSchedules().append(pop.getSchedules()[i])
                j += 1
        roulettePop.getSchedules().sort(key=lambda x: x.getFitness(), reverse = True)
        return roulettePop


class Student:
    def __init__(self, name, course):
        self._name = name
        self._course = course   
    def getName(self):
        return self._name
    def getCourse(self):
        return self._course
class Exam:
    def __init__(self, id, course,courseCode):
        self._course = course
        self.courseCode = courseCode
        self._id = id
        self._day = None
        self._examTime = None
        self._room = None
    def getCourse(self):
        return self._course
    def getcourseCode(self):
        return self.courseCode
    def getID(self):
        return self._id   
    def getTeachers(self):
        return data.getTeacher()
    def getStudent(self):
        return data.getStudent()
    def getExamTime(self):
        return self._examTime
    def getRoom(self):
        return data.getRoom()
    def getDay(self):
        return self._day
    def setExamTime(self, examTime):
        self._examTime = examTime
    def setRoom(self, room):
        self._room = room
    def setDay(self, day):
        self._day = day
    def __str__(self):
        return self._name

class Course:
    def __init__(self, name, code, allowedRooms):
        self._name = name
        self._code = code
        self._allowedRooms = allowedRooms
        
    def getName(self):
        return self._name
    def getCode(self):
        return self._code
    def getAllowedRooms(self):
        return self._allowedRooms
    def __str__(self):
        return self._name
    
class DisplayStuff:
    def printData(self):
        print ("All available (input) data")
        self.printInpData()
        
    #..................printing input data in pretty table.....................
    def printInpData(self):
        student = data.getStudent()
        k = 1
        inpDataTable = prettytable.PrettyTable(['Exam #', 'Teacher', 'Course', 'Student', 'Duration', 'Allowed Classrooms'])
        for i in range(0, len(student)):
            courses = student[i].getCourse()
            course = data.getCourse()
            for j in range(0, len(courses)):
                if courses== course[j].getCode():
                    inpDataTable.add_row([str(k), data.getTeacher(), courses, student[i].getName(), '3 hrs',course[j].getAllowedRooms()] )
                    k+=1
        print(inpDataTable)
 
    #..................printing generations in pretty tables...................
    def printGen(self, population):
        schedules = population.getSchedules()
        students =data.getStudent()
        for i in range(0, len(schedules)):
            print("\nChromosome ", i+1, " Fitness: ", round(schedules[i].getFitness(), 3), " Conflicts: ", schedules[i].getConflict())
            table1 = prettytable.PrettyTable(['Day', 'Time', 'Course', 'Teacher', 'Room'])
            schedules[i].sortDays()
            sortedDays = schedules[i].getSorted()
            for j in range(0, len(sortedDays)):
                exams = sortedDays[j]
                for k in range(0, len(exams)):
                    #,exams[k].getStudent()[k].getName()
                    table1.add_row([exams[k].getDay(), exams[k].getExamTime(), exams[k].getCourse(), exams[k].getTeachers(), exams[k].getRoom()])
            print(table1)

#...................Driver Code....................
data = InputData()
displayStuff = DisplayStuff()
displayStuff.printData()
generationNumber = 0
print ("\n Generation No. " + str(generationNumber))
population = Population(population_size)
population.getSchedules().sort(key=lambda x: x.getFitness(), reverse = True)
displayStuff.printGen(population)
geneticAlgo = GeneticAlgorithm()
while (population.getSchedules()[0].getFitness() != 1.0 or population.getSchedules()[0].getConflict != 0):
    if population.getSchedules()[0].getFitness() == 1.0 or population.getSchedules()[0].getConflict == 0:
        print ("\n Generation # " + str(generationNumber+1))
        displayStuff.printGen(population)
        break
        
    generationNumber += 1
    print ("\n Generation No. " + str(generationNumber))
    population = geneticAlgo.evolve(population)
    population.getSchedules().sort(key=lambda x: x.getFitness(), reverse = True)
    displayStuff.printGen(population)
    
print("\n.............Hard constraints............")
print("Conflict handled: An exam will be scheduled for each course.")
print("Conflict handled: A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.")
print("Conflict handled: Exam will not be held on weekends.")
print("Conflict handled: Each exam must be held between 9 am and 5 pm.")
print("Conflict handled: Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.")
print("Conflict handled: A teacher cannot invigilate two exams in a row.\n")
 
print(".............Soft constraints.............")
print("Conflict handled: All students and teachers shall be given a break on Friday from 1-2.")
print("Conflict handled: A student shall not give more than 1 exam consecutively.")
print("Conflict handled: Two hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot so the faculty meetings shall be held in parts as they are now.\n")
print("\n\n")

All available (input) data
+--------+-------------------+--------+--------------------------------+----------+--------------------+
| Exam # |      Teacher      | Course |            Student             | Duration | Allowed Classrooms |
+--------+-------------------+--------+--------------------------------+----------+--------------------+
|   1    |    Usman Rashid   | SE110  |          Yasmin Ahmed          |  3 hrs   |        C307        |
|   2    |    Noreen Jamil   | CS211  |         Sara Zamberlan         |  3 hrs   |        C308        |
|   3    |    Kashif Munir   | CS217  |        Adam N Starling         |  3 hrs   |        C307        |
|   4    |   Aqeel Shahzad   | CS211  |          Nabila Altaf          |  3 hrs   |        C308        |
|   5    |    Kashif Munir   | SE110  |           Ayan Lowe            |  3 hrs   |        C307        |
|   6    |   Sehrish Hassan  | SE110  |          Adam L Finch          |  3 hrs   |        C307        |
|   7    |   Khadija Farooq 


 Generation No. 1

Chromosome  1  Fitness:  0.286  Conflicts:  2.5
+-----------+------------+---------------------------------------------+--------------------+------+
|    Day    |    Time    |                    Course                   |      Teacher       | Room |
+-----------+------------+---------------------------------------------+--------------------+------+
|   Monday  | 9:00-12:00 |             Software Engineering            |    Zohaib Iqbal    | C309 |
|   Monday  | 9:00-12:00 |  Computer Organization & Assembly Language  |    Mehboobullah    | C301 |
|   Monday  | 9:00-12:00 |           Programming Fundamentals          |    Zainab Abaid    | C303 |
|   Monday  | 9:00-12:00 |  Computer Organization & Assembly Language  |    Sanaa Ilyas     | C310 |
|   Monday  | 9:00-12:00 |     Communication & Presentation Skills     |    Zainab Moin     | C305 |
|   Monday  | 9:00-12:00 |  Computer Organization & Assembly Language  |     Amna Irum      | C303 |
|   Monday  | 2:00-5:00


Chromosome  1  Fitness:  0.4  Conflicts:  1.5
+-----------+------------+---------------------------------------------+--------------------+------+
|    Day    |    Time    |                    Course                   |      Teacher       | Room |
+-----------+------------+---------------------------------------------+--------------------+------+
|   Monday  | 9:00-12:00 |             Discrete Structures             |    Waqas Munir     | C310 |
|   Monday  | 9:00-12:00 |           Programming Fundamentals          |   Behjat Zuhaira   | C309 |
|   Monday  | 9:00-12:00 |          Fundamentals of Management         |    Gul e Aisha     | C303 |
|   Monday  | 9:00-12:00 |          Probability and Statistics         |    Sadia Nauman    | C302 |
|   Monday  | 2:00-5:00  | Computer Organization and Assembly Language |   Zeeshan Qaiser   | C301 |
|   Monday  | 2:00-5:00  |        Islamic and Religious Studies        |   Tayyab Nadeem    | C306 |
|   Monday  | 2:00-5:00  |               Dat


Chromosome  1  Fitness:  0.286  Conflicts:  2.5
+-----------+------------+---------------------------------------------+-----------------+------+
|    Day    |    Time    |                    Course                   |     Teacher     | Room |
+-----------+------------+---------------------------------------------+-----------------+------+
|   Monday  | 9:00-12:00 |  Computer Organization & Assembly Language  |  Zeeshan Qaiser | C305 |
|   Monday  | 9:00-12:00 |           Programming Fundamentals          |   Zainab Abaid  | C307 |
|   Monday  | 9:00-12:00 |     Communication & Presentation Skills     |   Gul e Aisha   | C306 |
|   Monday  | 2:00-5:00  |               Database Systems              |   Sadia Nauman  | C301 |
|   Monday  | 2:00-5:00  |               Database Systems              |   Shafaq Riaz   | C309 |
|   Monday  | 2:00-5:00  | Computer Organization and Assembly Language |    Hamda Khan   | C307 |
|   Monday  | 2:00-5:00  |             Digital Logic Design          


Chromosome  1  Fitness:  0.4  Conflicts:  1.5
+-----------+------------+---------------------------------------------+--------------------+------+
|    Day    |    Time    |                    Course                   |      Teacher       | Room |
+-----------+------------+---------------------------------------------+--------------------+------+
|   Monday  | 9:00-12:00 |             Discrete Structures             |   Behjat Zuhaira   | C306 |
|   Monday  | 9:00-12:00 |           Programming Fundamentals          |   Tayyaba Zainab   | C304 |
|   Monday  | 9:00-12:00 |          Fundamentals of Management         |   Ameen Chilwan    | C304 |
|   Monday  | 9:00-12:00 |          Probability and Statistics         |    Irum Inayat     | C308 |
|   Monday  | 2:00-5:00  |        Islamic and Religious Studies        |    Arshad Islam    | C306 |
|   Monday  | 2:00-5:00  | Computer Organization and Assembly Language |    Zainab Abaid    | C302 |
|   Monday  | 2:00-5:00  |        Islamic an

...


Chromosome  1  Fitness:  0.286  Conflicts:  2.5
+-----------+------------+---------------------------------------------+--------------------+------+
|    Day    |    Time    |                    Course                   |      Teacher       | Room |
+-----------+------------+---------------------------------------------+--------------------+------+
|   Monday  | 9:00-12:00 |                  Psychology                 |    Umair Arshad    | C308 |
|   Monday  | 9:00-12:00 |               Data Structures               |    Sadia Nauman    | C304 |
|   Monday  | 9:00-12:00 |             Software Engineering            |    Sadia Nauman    | C308 |
|   Monday  | 9:00-12:00 |              Operating Systems              |    Usman Rashid    | C305 |
|   Monday  | 9:00-12:00 |     Communication & Presentation Skills     | Farah Jabeen Awan  | C309 |
|   Monday  | 2:00-5:00  |           Programming Fundamentals          |    Shafaq Riaz     | C308 |
|   Monday  | 2:00-5:00  |               P


Chromosome  1  Fitness:  0.4  Conflicts:  1.5
+-----------+------------+---------------------------------------------+-------------------+------+
|    Day    |    Time    |                    Course                   |      Teacher      | Room |
+-----------+------------+---------------------------------------------+-------------------+------+
|   Monday  | 9:00-12:00 |                  Psychology                 |    Sumera Abbas   | C307 |
|   Monday  | 9:00-12:00 |               Data Structures               |  Maimoona Rassol  | C302 |
|   Monday  | 9:00-12:00 |             Software Engineering            |    Sadia Nauman   | C306 |
|   Monday  | 9:00-12:00 | Computer Organization and Assembly Language |   Hassan Mustafa  | C309 |
|   Monday  | 9:00-12:00 |               Data Structures               |    Sumera Abbas   | C302 |
|   Monday  | 9:00-12:00 |              Digital Forensics              |    Waqas Munir    | C306 |
|   Monday  | 9:00-12:00 |             Software Engin


Chromosome  1  Fitness:  0.4  Conflicts:  1.5
+-----------+------------+-------------------------------------------+-----------------+------+
|    Day    |    Time    |                   Course                  |     Teacher     | Room |
+-----------+------------+-------------------------------------------+-----------------+------+
|   Monday  | 9:00-12:00 |                 Psychology                |   Umair Arshad  | C301 |
|   Monday  | 9:00-12:00 |              Database Systems             |  Sehrish Hassan | C309 |
|   Monday  | 9:00-12:00 |            Software Engineering           |  Rohail Gulbaz  | C308 |
|   Monday  | 9:00-12:00 |         Fundamentals of Management        |   Gul e Aisha   | C310 |
|   Monday  | 9:00-12:00 |      Design & Analysis of Algorithms      |    Amna Irum    | C308 |
|   Monday  | 9:00-12:00 |             Operating Systems             |   Umair Arshad  | C302 |
|   Monday  | 9:00-12:00 |             Digital Forensics             |   Noor ul Ain   | 


Chromosome  1  Fitness:  0.222  Conflicts:  3.5
+-----------+------------+---------------------------------------------+--------------------+------+
|    Day    |    Time    |                    Course                   |      Teacher       | Room |
+-----------+------------+---------------------------------------------+--------------------+------+
|   Monday  | 9:00-12:00 |                  Psychology                 |    Naveed Ahmad    | C306 |
|   Monday  | 9:00-12:00 |             Marketing Management            | Farah Jabeen Awan  | C301 |
|   Monday  | 9:00-12:00 |             Software Engineering            |   Muhammad Asim    | C306 |
|   Monday  | 9:00-12:00 |             Digital Logic Design            |   Maheen Arshad    | C302 |
|   Monday  | 9:00-12:00 | Computer Organization and Assembly Language |   Faisal Cheema    | C303 |
|   Monday  | 9:00-12:00 |          Fundamentals of Management         |  Maimoona Rassol   | C304 |
|   Monday  | 9:00-12:00 |              Op


Chromosome  2  Fitness:  0.182  Conflicts:  4.5
+-----------+------------+---------------------------------------------+-------------------+------+
|    Day    |    Time    |                    Course                   |      Teacher      | Room |
+-----------+------------+---------------------------------------------+-------------------+------+
|   Monday  | 9:00-12:00 |                  Psychology                 |    Usman Ashraf   | C303 |
|   Monday  | 9:00-12:00 |               Data Structures               |   Waseem Shahzad  | C302 |
|   Monday  | 9:00-12:00 |             Software Engineering            |    Noreen Jamil   | C309 |
|   Monday  | 9:00-12:00 |             Discrete Structures             |   Khadija Farooq  | C309 |
|   Monday  | 9:00-12:00 | Computer Organization and Assembly Language |    Kashif Munir   | C310 |
|   Monday  | 9:00-12:00 |              Digital Forensics              |    Noreen Jamil   | C304 |
|   Monday  | 2:00-5:00  |            Differential 